# 🎵 AI Music Recommendation System - Google Colab

This notebook runs the complete Music AI Recommendation System with GPT-OSS-20B and neural networks.

## 🚀 Features:
- AI-powered music recommendations
- GPT-OSS-20B natural language explanations
- Spotify integration with OAuth
- RLHF preference learning
- Interactive Streamlit frontend

## 📋 Setup Instructions:
1. **Enable GPU**: Runtime > Change runtime type > GPU (T4 or better)
2. **Run all cells** in order
3. **Get Spotify API credentials** from https://developer.spotify.com/dashboard
4. **Update credentials** in the config cell
5. **Enjoy your AI music recommendations!**

---


## 🔧 Step 1: Install Dependencies and Setup


In [1]:
# Install required packages
%pip install -q streamlit fastapi uvicorn[standard] pydantic pydantic-settings
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -q transformers vllm scikit-learn numpy pandas
%pip install -q spotipy sqlalchemy aiosqlite httpx aiohttp requests
%pip install -q jedi>=0.16
%pip install -q plotly rich python-dotenv google-colab
%pip install -q pyngrok

print("✅ Dependencies installed successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 134.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5

In [2]:
# Clone the repository
!git clone https://github.com/rsm-wew068/hack-mvp.git
%cd hack-mvp

# Create necessary directories
!mkdir -p models data logs

print("✅ Repository cloned and directories created!")


Cloning into 'hack-mvp'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 47 (delta 5), reused 42 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 68.86 KiB | 11.48 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/hack-mvp
✅ Repository cloned and directories created!


In [3]:
# Create necessary directories
!mkdir -p models data logs

print("✅ Repository cloned and directories created!")


✅ Repository cloned and directories created!


In [4]:
# Setup ngrok for Colab port forwarding (alternative method)
from pyngrok import ngrok
import getpass

# Get ngrok authtoken (optional but recommended)
print("🔗 Setting up ngrok for better port forwarding...")
print("   You can get a free ngrok authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken")
print("   (Optional - the app will work without it, but ngrok provides better URLs)")

# Uncomment and add your ngrok authtoken if you have one:
ngrok.set_auth_token("32QgKuZ26X0TdHnYX6Whl2rg3tI_6UGuUxUD7uJ7ph623sfou")

print("✅ ngrok setup complete!")


🔗 Setting up ngrok for better port forwarding...
   You can get a free ngrok authtoken from: https://dashboard.ngrok.com/get-started/your-authtoken
   (Optional - the app will work without it, but ngrok provides better URLs)
✅ ngrok setup complete!


In [5]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 GPU detected: {gpu_name} (Count: {gpu_count})")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected. Please enable GPU in Runtime > Change runtime type > GPU")
    print("   The system will still work but with slower performance")


🚀 GPU detected: NVIDIA L4 (Count: 1)
💾 GPU Memory: 23.8 GB


In [6]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 GPU detected: {gpu_name} (Count: {gpu_count})")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU detected. Please enable GPU in Runtime > Change runtime type > GPU")
    print("   The system will still work but with slower performance")


🚀 GPU detected: NVIDIA L4 (Count: 1)
💾 GPU Memory: 23.8 GB


## 🔐 Step 2: Configure Spotify API Credentials


In [7]:
# Configure Spotify API credentials
import os

# Get your credentials from https://developer.spotify.com/dashboard
# 1. Create a new app
# 2. Add redirect URI: https://colab.research.google.com/callback
# 3. Copy Client ID and Client Secret here

SPOTIFY_CLIENT_ID = "b2731a573c9c47fa8be846d383294924"  # Replace with your actual Client ID
SPOTIFY_CLIENT_SECRET = "55f1854cebb9416b85a4c91914974cf0"  # Replace with your actual Client Secret

# Set environment variables
os.environ["SPOTIFY_CLIENT_ID"] = SPOTIFY_CLIENT_ID
os.environ["SPOTIFY_CLIENT_SECRET"] = SPOTIFY_CLIENT_SECRET
os.environ["SPOTIFY_REDIRECT_URI"] = "https://colab.research.google.com/callback"

# Colab-specific settings
os.environ["PYTHONPATH"] = "/content/hack-mvp"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["VLLM_HOST"] = "0.0.0.0"
os.environ["VLLM_PORT"] = "8002"
os.environ["API_BASE_URL"] = "http://localhost:8000"
os.environ["FRONTEND_URL"] = "http://localhost:8501"
os.environ["DATABASE_URL"] = "sqlite:///./data/music_app.db"
os.environ["MODEL_STORAGE_PATH"] = "./models"
os.environ["DATA_STORAGE_PATH"] = "./data"

print("✅ Environment configured!")
print(f"📱 Spotify Client ID: {SPOTIFY_CLIENT_ID[:10]}..." if SPOTIFY_CLIENT_ID != "your_client_id_here" else "⚠️  Please update your Spotify credentials above")


✅ Environment configured!
📱 Spotify Client ID: b2731a573c...


## 🧠 Step 3: Initialize AI Models


In [8]:
# Initialize neural network models
import torch
import sys
sys.path.append('/content/hack-mvp')

from models.neural_networks import MusicEmbeddingNet, BradleyTerryModel, DeepCollaborativeFilter

print("🧠 Initializing neural network models...")

# Create models
audio_model = MusicEmbeddingNet(num_audio_features=13, embedding_dim=128)
bt_model = BradleyTerryModel(num_items=100000, embedding_dim=64)
cf_model = DeepCollaborativeFilter(num_users=10000, num_items=100000, embedding_dim=128)

# Save models
torch.save(audio_model.state_dict(), 'models/audio_embedding.pth')
torch.save(bt_model.state_dict(), 'models/bradley_terry.pth')
torch.save(cf_model.state_dict(), 'models/collaborative_filter.pth')

print("✅ Neural network models initialized and saved!")
print(f"📊 Model sizes:")
print(f"   Audio Embedding: {sum(p.numel() for p in audio_model.parameters()):,} parameters")
print(f"   Bradley-Terry: {sum(p.numel() for p in bt_model.parameters()):,} parameters")
print(f"   Collaborative Filter: {sum(p.numel() for p in cf_model.parameters()):,} parameters")


🧠 Initializing neural network models...
✅ Neural network models initialized and saved!
📊 Model sizes:
   Audio Embedding: 25,728 parameters
   Bradley-Terry: 6,400,065 parameters
   Collaborative Filter: 14,113,025 parameters


## 🚀 Step 4: Start All Services


In [9]:
# Start all services in background
import subprocess
import threading
import time
import requests
from google.colab import output

print("🚀 Starting Music AI Recommendation System...")

# Enable Colab port forwarding
output.enable_custom_widget_manager()

def start_vllm_server():
    """Start vLLM server with GPT-OSS-20B."""
    print("🧠 Starting vLLM server with GPT-OSS-20B...")
    cmd = [
        "python", "-m", "vllm.entrypoints.api_server",
        "--model", "openai-community/gpt-oss-20b",
        "--host", "0.0.0.0",
        "--port", "8002",
        "--gpu-memory-utilization", "0.8",
        "--max-model-len", "4096"
    ]
    subprocess.run(cmd, capture_output=True)

def start_backend():
    """Start FastAPI backend."""
    print("⚡ Starting FastAPI backend...")
    cmd = [
        "python", "-m", "uvicorn",
        "backend.main:app",
        "--host", "0.0.0.0",
        "--port", "8000"
    ]
    subprocess.run(cmd, capture_output=True)

def start_frontend():
    """Start Streamlit frontend."""
    print("🖥️  Starting Streamlit frontend...")
    cmd = [
        "python", "-m", "streamlit", "run",
        "frontend/app.py",
        "--server.port", "8501",
        "--server.address", "0.0.0.0",
        "--server.headless", "true"
    ]
    subprocess.run(cmd, capture_output=True)

# Start services in background threads
vllm_thread = threading.Thread(target=start_vllm_server, daemon=True)
backend_thread = threading.Thread(target=start_backend, daemon=True)
frontend_thread = threading.Thread(target=start_frontend, daemon=True)

vllm_thread.start()
backend_thread.start()
frontend_thread.start()

print("⏳ Services starting... This may take a few minutes for the first run.")
print("   - vLLM server: Downloading GPT-OSS-20B model (~2-3 minutes)")
print("   - Backend: Starting FastAPI server")
print("   - Frontend: Starting Streamlit app")


🚀 Starting Music AI Recommendation System...
🧠 Starting vLLM server with GPT-OSS-20B...
⚡ Starting FastAPI backend...
🖥️  Starting Streamlit frontend...
⏳ Services starting... This may take a few minutes for the first run.
   - vLLM server: Downloading GPT-OSS-20B model (~2-3 minutes)
   - Backend: Starting FastAPI server
   - Frontend: Starting Streamlit app


## ⏳ Step 5: Wait for Services to Start


In [10]:
# Wait for services to be ready and test them
import time
import requests

print("⏳ Waiting for services to start...")
print("   This may take 3-5 minutes for the first run (model download)")

# Wait for initial startup
time.sleep(30)

# Test services
def test_service(url, name, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        if response.status_code == 200:
            print(f"✅ {name}: Ready (Status: {response.status_code})")
            return True
        else:
            print(f"⚠️  {name}: Status {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"⏳ {name}: Still starting... ({str(e)[:50]}...)")
        return False

# Test all services
services = [
    ("http://localhost:8000/health", "Backend API"),
    ("http://localhost:8002/health", "vLLM Server"),
    ("http://localhost:8501/_stcore/health", "Streamlit Frontend")
]

all_ready = False
attempts = 0
max_attempts = 20

while not all_ready and attempts < max_attempts:
    attempts += 1
    print(f"\n🔄 Attempt {attempts}/{max_attempts}:")

    results = []
    for url, name in services:
        results.append(test_service(url, name))

    all_ready = all(results)

    if not all_ready:
        print(f"⏳ Waiting 15 seconds before next attempt...")
        time.sleep(15)

if all_ready:
    print("\n🎉 All services are ready!")
else:
    print("\n⚠️  Some services may still be starting. You can continue or restart the services.")


⏳ Waiting for services to start...
   This may take 3-5 minutes for the first run (model download)

🔄 Attempt 1/20:
⏳ Backend API: Still starting... (HTTPConnectionPool(host='localhost', port=8000): M...)
⏳ vLLM Server: Still starting... (HTTPConnectionPool(host='localhost', port=8002): M...)
✅ Streamlit Frontend: Ready (Status: 200)
⏳ Waiting 15 seconds before next attempt...

🔄 Attempt 2/20:
⏳ Backend API: Still starting... (HTTPConnectionPool(host='localhost', port=8000): M...)
⏳ vLLM Server: Still starting... (HTTPConnectionPool(host='localhost', port=8002): M...)
✅ Streamlit Frontend: Ready (Status: 200)
⏳ Waiting 15 seconds before next attempt...

🔄 Attempt 3/20:
⏳ Backend API: Still starting... (HTTPConnectionPool(host='localhost', port=8000): M...)
⏳ vLLM Server: Still starting... (HTTPConnectionPool(host='localhost', port=8002): M...)
✅ Streamlit Frontend: Ready (Status: 200)
⏳ Waiting 15 seconds before next attempt...

🔄 Attempt 4/20:
⏳ Backend API: Still starting... (HTTPCon

## 🎵 Step 6: Access Your Music AI App


In [11]:
# Display access information
from IPython.display import HTML, display

print("🎉 Music AI Recommendation System is now running!")
print("\n📱 Access your application:")
print("\n🖥️  Frontend (Streamlit App):")
print("   The Streamlit app should be embedded below or accessible through Colab's interface")

# Try to embed the Streamlit app
try:
    from google.colab import output
    output.serve_kernel_port_as_iframe(8501, path='/', anchor_text='🎵 Open Music AI App')
    print("   ✅ Click the link above to open the app!")
except:
    print("   📱 App is running on http://localhost:8501")

print("\n⚡ Backend API:")
print("   http://localhost:8000")
print("\n📚 API Documentation:")
print("   http://localhost:8000/docs")

print("\n🧠 vLLM Server (GPT-OSS-20B):")
print("   http://localhost:8002")

print("\n🎯 What you can do:")
print("   1. 🔐 Authenticate with Spotify")
print("   2. 🎵 Get AI-powered music recommendations")
print("   3. 🎯 Train the AI with A/B comparisons")
print("   4. 👤 View your musical taste profile")
print("   5. 📊 Explore analytics and insights")


🎉 Music AI Recommendation System is now running!

📱 Access your application:

🖥️  Frontend (Streamlit App):
   The Streamlit app should be embedded below or accessible through Colab's interface
   📱 App is running on http://localhost:8501

⚡ Backend API:
   http://localhost:8000

📚 API Documentation:
   http://localhost:8000/docs

🧠 vLLM Server (GPT-OSS-20B):
   http://localhost:8002

🎯 What you can do:
   1. 🔐 Authenticate with Spotify
   2. 🎵 Get AI-powered music recommendations
   3. 🎯 Train the AI with A/B comparisons
   4. 👤 View your musical taste profile
   5. 📊 Explore analytics and insights


## 🎮 Step 7: Interactive Demo


In [12]:
# Run the interactive demo
import asyncio
import sys
sys.path.append('/content/hack-mvp')

from demo.showcase import HackathonDemo

print("🎮 Running interactive demo...")
print("   This showcases all the features of the Music AI system")

demo = HackathonDemo()
await demo.run_full_demo()


🎮 Running interactive demo...
   This showcases all the features of the Music AI system


╭──────────────────────────────────────────╮
│ 🎵 AI Music Recommendation System        │
│ Powered by GPT-OSS-20B & Neural Networks │
│ OpenAI Hackathon 2025                    │
╰──────────────────────────────────────────╯

1. 🔐 Spotify Authentication & Personal Data Access

Output()

         Sample User Playlists         
┏━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┓
┃ Playlist      ┃ Tracks ┃ Avg Energy ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━┩
│ Morning Vibes │     47 │       0.73 │
│ Workout Hits  │     32 │       0.91 │
│ Chill Evening │     28 │       0.42 │
└───────────────┴────────┴────────────┘

...

╭───────────────────────────────────────── 🎵 Industry Baby - Lil Nas X ──────────────────────────────────────────╮
│ Based on your hip-hop preferences and high-energy workout selections, this track combines catchy hooks with     │
│ confident delivery. The production quality and mainstream appeal match songs you've previously rated highly.    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

5. 🔄 Continuous Learning & Adaptation

                            AI Learning Progress                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Metric                  ┃ Before Training ┃ After Training ┃ Improvement ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Recommendation Accuracy │             67% │            84% │        +17% │
│ User Satisfaction       │           3.2/5 │          4.1/5 │        +28% │
│ Playlist Match Score    │            0.73 │           0.89 │        +22% │
│ Explanation Quality     │           3.5/5 │          4.3/5 │        +23% │
└─────────────────────────┴─────────────────┴────────────────┴─────────────┘

✨ The AI continuously improves with each user interaction!

╭─────────────────────────────────────────────────────────────────────────╮
│ 🏆 Demo Complete!                                                       │
│ This showcases creative use of GPT-OSS-20B for music understanding      │
│ Combined with neural networks and RLHF for personalized recommendations │
╰─────────────────────────────────────────────────────────────────────────╯

## 🎉 Congratulations!

You've successfully set up and running the **AI Music Recommendation System** with:

- ✅ **GPT-OSS-20B** for natural language explanations
- ✅ **Neural Networks** for personalized recommendations
- ✅ **RLHF Training** with interactive A/B comparisons
- ✅ **Spotify Integration** with OAuth authentication
- ✅ **Real-time Learning** from user feedback
- ✅ **Beautiful UI** with analytics and insights

### 🎯 Next Steps:
1. **Authenticate with Spotify** in the app
2. **Get AI recommendations** for your playlists
3. **Train the AI** using A/B comparisons
4. **Explore your musical taste profile**
5. **Share your experience** with others!

### 🚀 Deploy to Production:
- Use the Docker setup for cloud deployment
- Scale with multiple GPU instances
- Add more AI models and features

**Enjoy your AI-powered music discovery! 🎵**
